In [ ]:
import sys
!{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 >/dev/null


In [ ]:
import os
import random
import string
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

In [ ]:
dkube_training_op           = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [ ]:
image = "ocdr/d3-datascience-pytorch-cpu:v1.6"
serving_image = "ocdr/pytorchserver:1.6"
training_program = "Img-DN"
dataset = "Img-DN"
model = "Img-DN"
training_script = "python image-denoising/model-care.py"
transformer_code="image-denoising/transformer.py"
user = os.getenv('USERNAME')
framework = "pytorch"
f_version =  "1.6"
output_mount_point = "/opt/dkube/output/"
input_mount_point = "/opt/dkube/input/"

In [ ]:
@kfp.dsl.pipeline(
    name='Img-Denoising',
    description='IMAGE-DENOISING EXAMPLE'
)
def image_denoising_pipeline(token):
    
        train       = dkube_training_op(auth_token = token, container = json.dumps({"image": image}),
                                    framework=framework, version=f_version,
                                    program=training_program, run_script=training_script,
                                    datasets=json.dumps([dataset]), outputs=json.dumps([model]),
                                    input_dataset_mounts=json.dumps([input_mount_point]),
                                    output_mounts=json.dumps([output_mount_point]))

       
        serving     = dkube_serving_op(auth_token = token, model = train.outputs['artifact'], device='cpu', 
                                    serving_image=json.dumps({"image": serving_image}),
                                    transformer_image=json.dumps({"image": image}),
                                    transformer_project=training_program,
                                    transformer_code=transformer_code).after(train)

### Create a Run

In [ ]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)
client.create_run_from_pipeline_func(image_denoising_pipeline, arguments={"token":existing_token})

### Compile and generate tar ball

In [ ]:
# compile
import tempfile
f = tempfile.NamedTemporaryFile(suffix=".zip", delete=False)
kfp.compiler.Compiler().compile(image_denoising_pipeline,f.name)

### Upload pipeline

In [ ]:
pipeline_id = client.get_pipeline_id("Img-Denoising")

if(pipeline_id != None):
    pipeline_version = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 7))
    new_pipeline = client.upload_pipeline_version(f.name,pipeline_version_name = pipeline_version, pipeline_id = pipeline_id)
else:
    pipeline = client.upload_pipeline(f.name, pipeline_name="Img-Denoising")
